In [34]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
from functools import lru_cache

In [ ]:
df = pd.read_csv("policekillings.csv")
df.shape

In [ ]:
# filter null city values
df = df[(df["City"].notnull()) & (df["State"].notnull())]
df.shape

In [ ]:
# test city conversions
address='Denver, CO, United States of America'
geolocator = Nominatim(user_agent="Your_Name")
location = geolocator.geocode(address, timeout = None)
print(location.address)
print((location.latitude, location.longitude))

In [ ]:
# gen coordinates
# memoization table --> geolocator calls limited by api refresh rate
m = {}
bad_adds = []
def gen_coord(row):
    print(len(m))
    address = row["City"] + ", " + row["State"] + ", United States of America" 
    if address in m:
        return m[address]
    else:
        location = geolocator.geocode(address)
        if location:
            m[address] = (location.latitude, location.longitude)
        else:
            print(address)
            bad_adds.append(address)
            m[address] = None
    return m[address]

df["coordinates"] = df[["City", "State"]].apply(gen_coord, axis = 1)
df.head()

In [ ]:
print(df["coordinates"][0][0])
df = df[df["coordinates"].notna()]
df["latitude"] = df.apply(lambda row: row["coordinates"][0], axis=1)
df["longitude"] = df.apply(lambda row: row["coordinates"][1], axis=1)

In [35]:
# now that all rows have coordinate positions convert back to csv for use
df.to_csv("police_cleaned.csv", index = False)

In [89]:
tmp = pd.read_csv("police_cleaned.csv")

In [90]:
print("unknown race count: " + str(len(tmp[tmp["Victim's race"] == "Unknown Race"].index)))
print("unknown age count: " + str(len(tmp[tmp["Victim's age"] == "Unknown"].index)))
print("non NA url: " + str(len(tmp[tmp["URL of image of victim"].notna()].index)))
print("unique names: " + str(len(tmp["Victim's name"].unique())-1))


unknown race count: 921
unknown age count: 336
non NA url: 4832
unique names: 8819


In [91]:
print(list(tmp.columns))

["Victim's name", "Victim's age", "Victim's gender", "Victim's race", 'URL of image of victim', 'Date of Incident (month/day/year)', 'Street Address of Incident', 'City', 'State', 'Zipcode', 'County', 'Agency responsible for death', 'ORI Agency Identifier (if available)', 'Cause of death', 'A brief description of the circumstances surrounding the death', 'Official disposition of death (justified or other)', 'Criminal Charges?', 'Link to news article or photo of official document', 'Symptoms of mental illness?', 'Armed/Unarmed Status', 'Alleged Weapon (Source: WaPo and Review of Cases Not Included in WaPo Database)', 'Alleged Threat Level (Source: WaPo)', 'Fleeing (Source: WaPo)', 'Body Camera (Source: WaPo)', 'WaPo ID (If included in WaPo database)', 'Off-Duty Killing?', 'Geography (via Trulia methodology based on zipcode population density: http://jedkolko.com/wp-content/uploads/2015/05/full-ZCTA-urban-suburban-rural-classification.xlsx )', 'MPV ID', 'Fatal Encounters ID', 'Encounter 

In [92]:
# create year column 
tmp["date"] = pd.to_datetime(tmp["Date of Incident (month/day/year)"])

In [93]:
tmp["year"] = tmp.apply(lambda row: row["date"].year, axis = 1)

In [95]:
tmp.to_csv("police_cleaned.csv", index = False)